# Masking a range

In this example, we will use a slider to mask data outside a selected range in a two-dimensional image.
The data inside the range will be summed along the vertical dimension,
and displayed as a one-dimensional plot below the image.

In [ ]:
%matplotlib widget
import plopp as pp
import scipp as sc
import numpy as np
import ipywidgets as ipw

We first generate some data that contains three bands of peaks that all have different spreads.

In [ ]:
npeaks = 200
per_peak = 500
spread = 30.
ny = 300
nx = 300
a = np.ones((ny, nx))

xbins = np.arange(nx + 1)
ybins = np.arange(ny + 1)
for n in range(npeaks):
    xc = np.random.random() * nx
    yc = np.random.choice([ny/4, ny/2, 3*ny/4])
    xy = np.random.normal(loc=(xc, yc),
                          scale=spread * np.random.random(),
                          size=[per_peak, 2])
    h, ye, xe = np.histogram2d(xy[:, 1], xy[:, 0], bins=(ybins, xbins))
    a += h

da = sc.DataArray(data=sc.array(dims=['y', 'x'], values=a, unit='counts'),
                  coords={'x': sc.array(dims=['x'], values=xbins, unit='cm'),
                          'y': sc.array(dims=['y'], values=ybins, unit='cm')}
                 )

We then construct our interface with a slider, a node that adds a mask, and a node that sums the unmasked data along the `y` dimension.

In [ ]:
ydim = 'y'

# A function that will add a mask outside of selected range
def add_mask(da, trunc_range):
    min_tr, max_tr = trunc_range
    out = da.copy(deep=False)
    out.masks[ydim] = sc.zeros(sizes={ydim: da.sizes[ydim]}, dtype=bool)
    out.masks[ydim].values[:min_tr] = True
    out.masks[ydim].values[max_tr:] = True
    return out

in_node = pp.input_node(da)
in_node.name = 'Input node'

# Make a range slider
slider = ipw.IntRangeSlider(max=da.sizes[ydim]-1,
                            description='y indices',
                            layout={'width': '400px'})

slider_node = pp.widget_node(slider)
slider_node.name = 'Slider node'

# Add mask
mask_node = pp.node(add_mask)(in_node, trunc_range=slider_node)
mask_node.name = 'Add masks'

# Node that sums along the y dimension
sum_node = pp.node(sc.sum, dim=ydim)(mask_node)
sum_node.name = 'Sum Y'

# Make figures
fig2d = pp.figure2d(mask_node, norm='log')
fig1d = pp.figure1d(sum_node, grid=True, ls='solid', marker='')
ipw.VBox([slider, fig2d, fig1d])

In [ ]:
pp.show_graph(in_node)

In [ ]:
# This cell is used to generate the thumbnail for the docs gallery.
# It is hidden from the online documentation.
fig2d.save('../../_static/gallery/masking-a-range-thumbnail.png')